# Perovskite Material Analysis and Band Gap Prediction

This notebook documents the workflow for analyzing perovskite material compositions and predicting their band gaps using machine learning models. The steps include data cleaning, feature extraction, and model training with Ridge Regression and Random Forest Regressor. Our methodology was designed to allow us to reproduce findings from the Hussain et al paper. 

## Data Cleaning and Preparation

In this section, we load a sample dataset and perform data cleaning and preparation. The steps include:
1. Loading the dataset into a pandas DataFrame.
2. Cleaning and normalizing molecule names.
3. Converting coefficients to floats.
4. Extracting ions and coefficients into dictionaries.
5. Identifying unique molecules and creating new columns for each unique molecule.
6. Calculating the proportions for each molecule and assigning them to the corresponding columns.
7. Dropping the original ion and coefficient columns as they are no longer needed.
8. Saving the cleaned and modified DataFrame to a new CSV file.
```

In [1]:

import pandas as pd
import re
# Load the CSV file
file_path = 'perovsite_database_query.csv'  # Change this to your actual file path
data = pd.read_csv(file_path)

# Columns to keep from the dataset
columns_to_keep = [
    'JV_default_Voc', 'JV_default_Jsc', 'JV_default_FF', 'JV_default_PCE', 
    'Perovskite_band_gap', 'Perovskite_composition_a_ions', 'Perovskite_composition_a_ions_coefficients',
    'Perovskite_composition_b_ions', 'Perovskite_composition_b_ions_coefficients',
    'Perovskite_composition_c_ions', 'Perovskite_composition_c_ions_coefficients'
]
data_cleaned = data[columns_to_keep]

# Drop rows with NaN values in important columns
data_cleaned.dropna(subset=[
    'Perovskite_composition_a_ions', 'Perovskite_composition_a_ions_coefficients',
    'Perovskite_composition_b_ions', 'Perovskite_composition_b_ions_coefficients',
    'Perovskite_composition_c_ions', 'Perovskite_composition_c_ions_coefficients'
], inplace=True)

# Remove rows where 'Perovskite_band_gap' contains '|'
data_cleaned = data_cleaned[~data_cleaned['Perovskite_band_gap'].str.contains('|', na=False)]

# Reset index after dropping rows
data_cleaned.reset_index(drop=True, inplace=True)

# Function to clean and normalize molecule names
def clean_molecule_name(name):
    # Remove leading/trailing spaces and normalize space
    name = name.strip()
    
    # Allow alphanumeric characters and hyphens; also preserve names in parentheses
    name = re.sub(r'[^a-zA-Z0-9\s\-()]+', ' ', name)

    # Replace multiple spaces with a single space and trim again
    name = re.sub(r'\s+', ' ', name).strip()
    
    # Split the string into individual molecule components based on spaces
    elements = name.split()

    # Include valid molecule names, allowing for bracketed entries
    elements = [element for element in elements if element and 
                not element.replace('.', '', 1).isdigit()]

    return elements

# Function to clean and convert coefficients to floats
def clean_and_convert_coefficient(coefficient):
    try:
        # Remove non-numeric characters except periods and minus signs
        cleaned_coefficient = re.sub(r'[^0-9.-]', '', coefficient)
        return float(cleaned_coefficient)
    except ValueError:
        return 0.0  # Default to 0.0 if conversion fails

# Function to split ions and coefficients into dictionaries
def extract_ions_and_coefficients(ions_column, coefficients_column):
    # Split ions and coefficients by ';' and '|'
    ions = re.split(r'[;|]', ions_column)
    coefficients = re.split(r'[;|]', coefficients_column)
    
    # Clean and convert to floats
    coefficients = [clean_and_convert_coefficient(c) for c in coefficients]
    
    # Clean ion names and split into individual elements
    all_ions = []
    for ion in ions:
        cleaned_ions = clean_molecule_name(ion)  # Clean and split the ion names
        all_ions.extend(cleaned_ions)            # Add each individual element
    
    return all_ions, coefficients

# Step 1: Identify unique molecules
unique_molecules = set()

# Go through each row to identify unique ions
for index, row in data_cleaned.iterrows():
    for column_group in ['a', 'b', 'c']:
        ions_column = f'Perovskite_composition_{column_group}_ions'
        coefficients_column = f'Perovskite_composition_{column_group}_ions_coefficients'
        
        ions = str(row[ions_column]).split(';')
        all_ions = []
        for ion in ions:
            cleaned_ions = clean_molecule_name(ion)
            all_ions.extend(cleaned_ions)
        unique_molecules.update(all_ions)

# Remove any 'nan' from unique molecules set
unique_molecules.discard('nan')

# Step 2: Create new columns for each unique molecule
for molecule in unique_molecules:
    data_cleaned[molecule] = 0.0  # Initialize columns for each molecule with 0.0

# Step 3: Calculate the proportions for each molecule
for index, row in data_cleaned.iterrows():
    # Iterate over the ion columns and their coefficients
    for column_group in ['a', 'b', 'c']:
        ions_column = f'Perovskite_composition_{column_group}_ions'
        coefficients_column = f'Perovskite_composition_{column_group}_ions_coefficients'
        
        ions, coefficients = extract_ions_and_coefficients(str(row[ions_column]), str(row[coefficients_column]))

        total_coeff = sum(coefficients) if sum(coefficients) != 0 else 1  # Avoid division by zero
        
        # Calculate proportion and assign to the corresponding molecule columns
        for ion, coeff in zip(ions, coefficients):
            data_cleaned.at[index, ion] += coeff / total_coeff  # Add the proportion to the column

# Step 4: Drop the original ion and coefficient columns as they are no longer needed
columns_to_drop = [
    'Perovskite_composition_a_ions', 'Perovskite_composition_a_ions_coefficients',
    'Perovskite_composition_b_ions', 'Perovskite_composition_b_ions_coefficients',
    'Perovskite_composition_c_ions', 'Perovskite_composition_c_ions_coefficients'
]
data_cleaned.drop(columns=columns_to_drop, inplace=True)
# Remove brackets from molecule names in the unique molecules set
unique_molecules_cleaned = {molecule.strip('()') for molecule in unique_molecules}

# Update the output to show the cleaned unique molecules
print("Cleaned Unique molecules identified:", unique_molecules_cleaned)

# Filter the unique molecules set to keep only the specified molecules
molecules_to_keep = {'I', 'Br', 'Pb', 'Sn', 'Cl', 'FA', 'MA', 'Cs'}
filtered_unique_molecules = unique_molecules.intersection(molecules_to_keep)

# Update the output to show the filtered unique molecules
print("Filtered Unique molecules identified:", filtered_unique_molecules)



# Drop all columns in data_cleaned that are not in molecules_to_keep
columns_to_drop = [col for col in data_cleaned.columns if col not in molecules_to_keep and col not in columns_to_keep]
data_cleaned.drop(columns=columns_to_drop, inplace=True)


# Update the output to show the remaining columns
print("Remaining columns after dropping:", data_cleaned.columns)
# Save the modified dataframe to a new CSV
output_file_path = 'modified_file.csv'
data_cleaned.to_csv(output_file_path, index=False)

print("CSV file modified and saved as:", output_file_path)
print("Unique molecules identified:", unique_molecules)


C:\Users\c\AppData\Local\Temp\ipykernel_31644\2672202543.py:5: DtypeWarning: Columns (10,22,29,31,32,35,36,40,44,45,46,48,51,54,65,84,89,90,93,98,99,100,105,108,115,118,122,123,125,130,134,138,142,143,144,146,149,152,163,166,167,171,172,173,175,178,181,192,194,225,271,272,273,277,304,315,321,325,330,331,335,336,342,348,369,371,373,374,376,380,384,387,403,405,407,409) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)
C:\Users\c\AppData\Local\Temp\ipykernel_31644\2672202543.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.dropna(subset=[
C:\Users\c\AppData\Local\Temp\ipykernel_31644\2672202543.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor p

Cleaned Unique molecules identified: {'MA', 'GA', 'Br', 'ALA', 'Ba', '3-Pr(NH3)2', 'IM', 'TMA', 'PDMA', 'FA', 'Mg', 'Na', 'Ti', 'OdA', 'CPEA', 'ThMA', 'Cs', 'GU', 'Ni', '1', 'CHMA', 'SCN', 'THM', 'A43', 'Br-PEA', 'BIM', 'Te', 'TFEA', 'OA', 'C8H17NH3', 'PEI', 'Cl-PEA', 'C4H9NH3', 'TBA', 'iso-BA', 'BYA', 'Au', 'I', 'TN', 'MIC1', 'PF6', 'HA', 'K', 'iPA', 'AVA', 'DMA', 'ODA', 'Ag', 'Ge', 'PR', 'Nb', 'oF1PEA', 'BA', 'HTAB', 'f-PEA', 'pF1PEA', 'PDA', 'PEA', 'F5PEA', 'Bi', 'Anyl', 'DI', 'Cl', 'Ca', 'Li', 'GABA', 'DA', 'FPEA', 'C4H9N2H6', 'EA', '3AMPY', 'Sb', 'Eu', 'APMim', 'BDA', '4FPEA', 'PA', 'BEA', 'BE', '3AMP', 'Hg', 'CH3ND3', 'Tb', 'Mn', 'C6H4NH2', '6-ACA', 'IA', 'EDA', 'BU', 'O', 'Rb', 'PBA', 'HEA', 'Sr', 'PTA', 'BI', 'Bn', 'MIC2', 'Aa', 'BzDA', 'HdA', 'Ada', 'CIEA', '5-AVAI', 'TA', 'Co', '4AMPY', 'Fe', 'DPA', 'BdA', 'PGA', 'n-C3H7NH3', 'FEA', 'EPA', 'BF4', 'Sn', '4ApyH', 'AN', 'HDA', 'HAD', 'EU-pyP', 'pFPEA', 'oFPEA', 'PPEA', 'DAT', 'TEA', 'CH3)3S', 'MTEA', 'IEA', 'PMA', 'PyEA', 'F-PEA

## Training and testing ML models

In this next part, we train and test different ML models (identical to those used in the Hussain et al paper) on the cleaned perovskite database. Following this, we predict values of band gap and PCE and compare the error between these predictions and real values seen in the database. 

In [11]:
###### CRISTINA TEST

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('modified_file.csv')

# Remove rows with NaN values (including np.nan)
data = data.dropna()  # This drops rows with any NaN values

# Prepare features and targets

# For prediction of each target variable y (Voc, Jsc, FF, PCE and band gap), we want X to exclude/drop that specific target variable, 
# but include the other (4) "y's" (and all other features/molecules), dont want to include certain feature in prediction of itself

# For band gap prediction
X = data.drop(['Perovskite_band_gap'], axis=1)
y_band_gap = data['Perovskite_band_gap']
# Split the data
X_train, X_test, y_band_gap_train, y_band_gap_test = train_test_split(X, y_band_gap, test_size=0.2, random_state=42)
# Initialise models 
models = {
    'LR': LinearRegression(),
    'RR': Ridge(),
    'KNN': KNeighborsRegressor(),
    'RF': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# train and evaluate models
results = {}
y_band_gap_pred = {}

######### CONTINUE HERE, check this
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Train and evaluate models
results = {}
y_band_gap_pred = {}

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_band_gap_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_band_gap_pred[name] = y_pred
    
    # Evaluate the model
    mae = mean_absolute_error(y_band_gap_test, y_pred)
    mse = mean_squared_error(y_band_gap_test, y_pred)
    r2 = r2_score(y_band_gap_test, y_pred)
    
    # Store the results in a dictionary
    results[name] = {
        'Mean absolute error': mae,
        'Mean square error': mse,
        'R2 Score': r2
    }

# Print the results
for model_name, metrics in results.items():
    print(f"{model_name} performance:")
    print(f"Mean absolute error: {metrics['Mean absolute error']}")
    print(f"Mean square error: {metrics['Mean square error']}")
    print(f"R2 Score: {metrics['R2 Score']}\n")

# Display predictions for band gap for each of the models

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame(y_band_gap_test.values, columns=['Actual'], index=y_band_gap_test.index)
# Add a title to the DataFrame
predictions_df.style.set_caption("Band Gap Predictions for Different ML Models")

# Add the predictions for each model to the DataFrame
for name, y_pred in y_band_gap_pred.items():
    predictions_df[name + '_Predicted'] = y_pred

# Display the first few rows of the DataFrame to see the results
print(predictions_df.head())


LR performance:
Mean absolute error: 0.02538766020300304
Mean square error: 0.00311387569708245
R2 Score: 0.7726241732513978

RR performance:
Mean absolute error: 0.025500870187326486
Mean square error: 0.003147480517194903
R2 Score: 0.770170342559644

KNN performance:
Mean absolute error: 0.023537444146559446
Mean square error: 0.0030909265272564797
R2 Score: 0.7742999262261434

RF performance:
Mean absolute error: 0.017787095906210037
Mean square error: 0.0020605546339697712
R2 Score: 0.8495378881377568

XGBoost performance:
Mean absolute error: 0.017895967415851442
Mean square error: 0.0020297801201198495
R2 Score: 0.8517850492996388

       Actual  LR_Predicted  RR_Predicted  KNN_Predicted  RF_Predicted  \
9004      1.6      1.591892      1.591924           1.61       1.59463   
36140     1.6      1.605958      1.606303           1.59       1.59100   
25301     1.6      1.591443      1.591481           1.56       1.59610   
26083     1.6      1.599469      1.599686           1.60  

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('modified_file.csv')

# Remove rows with NaN values (including np.nan)
data = data.dropna()  # This drops rows with any NaN values

# Prepare features and targets

# For prediction of each target variable y (Voc, Jsc, FF, PCE and band gap), we want X to exclude/drop that specific target variable, 
# but include the other (4) "y's" (and all other features/molecules)

X = data.drop(['JV_default_Voc', 'JV_default_Jsc', 'JV_default_FF', 'JV_default_PCE', 'Perovskite_band_gap'], axis=1)
y_band_gap = data['Perovskite_band_gap']
y_pce = data['JV_default_PCE']
y_voc = data['JV_default_Voc']
y_jsc = data['JV_default_Jsc']
y_ff = data['JV_default_FF']

# Split the data
X_train, X_test, y_band_gap_train, y_band_gap_test, y_pce_train, y_pce_test, \
y_voc_train, y_voc_test, y_jsc_train, y_jsc_test, y_ff_train, y_ff_test = \
    train_test_split(X, y_band_gap, y_pce, y_voc, y_jsc, y_ff, test_size=0.2, random_state=42)

# Initialize models
models = {
    'LR': LinearRegression(),
    'RR': Ridge(),
    'KNN': KNeighborsRegressor(),
    'RF': RandomForestRegressor(random_state=42),
    'XGBoost': XGBRegressor(random_state=42)
}

# Function to evaluate models
def evaluate_model(y_true, y_pred):
    r_value = np.sqrt(r2_score(y_true, y_pred))
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return r_value, rmse

# Train and evaluate models
results = {model_name: {'Band gap prediction': [], 'PCE direct prediction': [], 'PCE calculated': []}
           for model_name in models.keys()}

for model_name, model in models.items():
    # Band gap prediction
    model.fit(X_train, y_band_gap_train)
    y_band_gap_pred = model.predict(X_test)
    r_value, rmse = evaluate_model(y_band_gap_test, y_band_gap_pred)
    results[model_name]['Band gap prediction'] = [r_value, rmse]

    # PCE direct prediction
    model.fit(X_train, y_pce_train)
    y_pce_pred = model.predict(X_test)
    r_value, rmse = evaluate_model(y_pce_test, y_pce_pred)
    results[model_name]['PCE direct prediction'] = [r_value, rmse]

    # Train models for Voc, Jsc, and FF
    model.fit(X_train, y_voc_train)
    y_voc_pred = model.predict(X_test)
    model.fit(X_train, y_jsc_train)
    y_jsc_pred = model.predict(X_test)
    model.fit(X_train, y_ff_train)
    y_ff_pred = model.predict(X_test)

    # Calculate PCE from predicted Voc, Jsc, and FF
    # NOTE: need to add a column for 'real' calculated PCE in the dataset (to avoid nan r values)?
    power_in = 100   # subject to change
    y_pce_calculated = y_voc_pred * y_jsc_pred * y_ff_pred / power_in
    r_value, rmse = evaluate_model(y_pce_test, y_pce_calculated)
    results[model_name]['PCE calculated'] = [r_value, rmse]

# Create a DataFrame for the results
df_results = pd.DataFrame({
    'Models': list(models.keys()),
    'Band gap prediction r-Value': [results[model]['Band gap prediction'][0] for model in models],
    'Band gap prediction RMSE (eV)': [results[model]['Band gap prediction'][1] for model in models],
    'PCE direct prediction r-Value': [results[model]['PCE direct prediction'][0] for model in models],
    'PCE direct prediction RMSE (%)': [results[model]['PCE direct prediction'][1] for model in models],
    'PCE calculated r-Value': [results[model]['PCE calculated'][0] for model in models],
    'PCE calculated RMSE (%)': [results[model]['PCE calculated'][1] for model in models]
})

# Display the results
print(df_results)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 6))
ax.axis('off')

# Create the table
table = ax.table(cellText=df_results.values,
                 colLabels=df_results.columns,
                 cellLoc='center',
                 loc='center')

# Set table properties
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.5)

# Add title
plt.title("Table 1: ML model types with corresponding results", fontsize=12, fontweight='bold', pad=20)

# Save the figure
plt.tight_layout()
plt.savefig('table_1_results.png', dpi=300, bbox_inches='tight')
plt.close()

print("Table 1 with results has been generated and saved as 'table_1_results.png'")

C:\Users\c\AppData\Local\Temp\ipykernel_31644\1732292398.py:41: RuntimeWarning: invalid value encountered in sqrt
  r_value = np.sqrt(r2_score(y_true, y_pred))
C:\Users\c\AppData\Local\Temp\ipykernel_31644\1732292398.py:41: RuntimeWarning: invalid value encountered in sqrt
  r_value = np.sqrt(r2_score(y_true, y_pred))
C:\Users\c\AppData\Local\Temp\ipykernel_31644\1732292398.py:41: RuntimeWarning: invalid value encountered in sqrt
  r_value = np.sqrt(r2_score(y_true, y_pred))
C:\Users\c\AppData\Local\Temp\ipykernel_31644\1732292398.py:41: RuntimeWarning: invalid value encountered in sqrt
  r_value = np.sqrt(r2_score(y_true, y_pred))
C:\Users\c\AppData\Local\Temp\ipykernel_31644\1732292398.py:41: RuntimeWarning: invalid value encountered in sqrt
  r_value = np.sqrt(r2_score(y_true, y_pred))
C:\Users\c\AppData\Local\Temp\ipykernel_31644\1732292398.py:41: RuntimeWarning: invalid value encountered in sqrt
  r_value = np.sqrt(r2_score(y_true, y_pred))


    Models  Band gap prediction r-Value  Band gap prediction RMSE (eV)  \
0       LR                     0.874396                       0.056782   
1       RR                     0.872853                       0.057106   
2      KNN                     0.913284                       0.047667   
3       RF                     0.924299                       0.044665   
4  XGBoost                     0.924254                       0.044677   

   PCE direct prediction r-Value  PCE direct prediction RMSE (%)  \
0                       0.365757                        4.651490   
1                       0.365795                        4.651415   
2                            NaN                        5.206572   
3                       0.413458                        4.550596   
4                       0.415590                        4.545743   

   PCE calculated r-Value  PCE calculated RMSE (%)  
0                     NaN                12.532712  
1                     NaN               

## Feature importance

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('modified_file.csv')
# Remove rows with NaN values (including np.nan)
data = data.dropna()  # This drops rows with any NaN values

# Prepare features and target
X = data.drop(['JV_default_Voc', 'JV_default_Jsc', 'JV_default_FF', 'JV_default_PCE', 'Perovskite_band_gap'], axis=1)
y = data['Perovskite_band_gap']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Ridge Regression model
rr_model = Ridge()
rr_model.fit(X_train, y_train)

# Train Random Forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importance for Ridge Regression
rr_importance = np.abs(rr_model.coef_)
rr_importance = rr_importance / np.sum(rr_importance)
rr_feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rr_importance})
rr_feature_importance = rr_feature_importance.sort_values('importance', ascending=False).head(8)

# Get feature importance for Random Forest
rf_importance = rf_model.feature_importances_
rf_feature_importance = pd.DataFrame({'feature': X.columns, 'importance': rf_importance})
rf_feature_importance = rf_feature_importance.sort_values('importance', ascending=False).head(8)

# Create subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot for RR model
ax1.barh(rr_feature_importance['feature'], rr_feature_importance['importance'], color='steelblue', height=0.6)
ax1.set_xlabel('Impact')
ax1.set_title('(a) RR model')
ax1.invert_yaxis()  # Invert y-axis to match the image

# Plot for RF model
ax2.barh(rf_feature_importance['feature'], rf_feature_importance['importance'], color='steelblue', height=0.6)
ax2.set_xlabel('Impact')
ax2.set_title('(b) RF model')
ax2.invert_yaxis()  # Invert y-axis to match the image

# Add overall title
fig.suptitle('Fig. 4 Impact of different materials on band gap prediction', fontsize=12, fontweight='bold')

# Adjust layout and save
plt.tight_layout()
plt.savefig('figure_4_results.png', dpi=300, bbox_inches='tight')
plt.close()

print("Figure 4 with results has been generated and saved as 'figure_4_results.png'")

# Print feature importances
print("Ridge Regression Feature Importance:")
print(rr_feature_importance)
print("\nRandom Forest Feature Importance:")
print(rf_feature_importance)

Figure 4 with results has been generated and saved as 'figure_4_results.png'
Ridge Regression Feature Importance:
  feature  importance
5      Cl    0.270267
1      Br    0.216369
6      Sn    0.182374
2      FA    0.116853
0      MA    0.083122
7      Pb    0.072064
3      Cs    0.047017
4       I    0.011934

Random Forest Feature Importance:
  feature  importance
4       I    0.583427
6      Sn    0.118893
7      Pb    0.101370
1      Br    0.070698
2      FA    0.048938
3      Cs    0.036947
0      MA    0.029220
5      Cl    0.010508
